In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
path = "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"

In [2]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
!mkdir fish_dataset

In [4]:
fishes = []
for name in os.listdir(path):
    if os.path.isdir(os.path.join(path, name)):
        fishes.append(name)

In [5]:
for fish in fishes:
    fish_image_folder = os.path.join(path, fish, fish)
    shutil.copytree(fish_image_folder, os.path.join("./fish_dataset", fish))

In [6]:
print(os.listdir("./fish_dataset"))
print(os.listdir("./fish_dataset/Gilt-Head Bream")[:3])

In [7]:
img_size = (224, 224)
batch_size = 32

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
)


In [ ]:
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # regularization
    layers.Dense(9, activation="softmax"),
])

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"],
)

In [ ]:
model.fit(ds_train, batch_size=64, epochs=5, verbose=2, validation_data=ds_validation)

In [16]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [8]:
from tensorflow.keras.applications import EfficientNetB0


inputs = layers.Input(shape=(224, 224, 3))
#x = img_augmentation(inputs)
outputs = EfficientNetB0(include_top=True, weights=None, classes=9)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()



In [ ]:
epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_validation, verbose=2)